In [10]:
base_path = '../autodl-tmp'

In [11]:
seeddata_name = 'poli'
# data_name = 'goss'

In [12]:
few_shot = 2
# few_shot = 4
# few_shot = 8
# few_shot = 16
# few_shot = 100

In [13]:
seed = 1
# seed = 2
# seed = 3
# seed = 4
# seed = 5

In [33]:
import torch
from datasets import load_dataset


#定义数据集
class Dataset(torch.utils.data.Dataset):
    def __init__(self, dataset_type):
        
        path = '{0}/{1}/{2}shot/seed{3}/{4}'.format(base_path, data_name, few_shot, seed, dataset_type)
        
        self.dataset = load_dataset(path=path, split=dataset_type)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['clean_text']
        label = self.dataset[i]['label']

        return text, label


dataset = Dataset('train')

# len(dataset), dataset[1]
dataset[1][0], dataset[1][1]

Using custom data configuration train-a9f345196c811a9a
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/train-a9f345196c811a9a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


('marco rubio portrayed fellow gop senator and presidential hopeful ted cruz as all talk and no action on defense in an interview on meet the press.  "he talks tough on some of these issues," rubio said dec. 13. "for example, he was going to carpet bomb isis. but the only budget he\'s ever voted for in his time in the senate is a budget that cut defense spending by more than barack obama proposes we cut it."  rubio has tried to portray himself as a strong supporter of the military, while cruz has tried to appeal to hawks as well as libertarians , who typically favor less defense spending.  we decided to fact-check rubios claim that cruz voted for a budget that cut defense spending by more than the democratic president.  cruzs vote on rand pauls budget proposal  rubios campaign pointed to cruzs 2013 vote in favor of a budget proposal by u.s. sen. rand paul, r-ky., who is also running for president. cruz was one of just 18 senators, all republican, to vote in favor of pauls amendment. ru

In [30]:
from transformers import RobertaTokenizer, RobertaModel

token = RobertaTokenizer.from_pretrained('roberta-base')

# #加载字典和分词工具
# token = BertTokenizer.from_pretrained('roberta-base')

token

PreTrainedTokenizer(name_or_path='roberta-base', vocab_size=50265, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})

In [42]:
dataset[0]

('    target ceo brian cornell announced today the retail giant will be discontinuing sales of the holy bible. the company has come under fire recently from religious groups for its new transgender bathroom policy. the king james bible has been available for purchase at target stores ever since the company was founded as goodfellow dry goods back in 1902.  speaking with cnbc, cornell cited recent protests from what he calls religious extremists as the reason to pull the bible from its shelves. target will no longer cater to religious extremists, said cornell. if that means removing the bible from our shelves, then so be it. cornell said the big box chain is sticking to its guns regarding their bathroom policy, and will be phasing out anything having to do with religion. we believe that everyone, every team member, every guest, and every community, deserves to be treated equally, regardless of their religious beliefs.  more  ~~~~~~~~~~~~~~~~~~~~~~~~  this is the video referenced above: 

In [62]:
def collate_fn(data):
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]

    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=500,
                                   return_tensors='pt',
                                   return_length=True,
                                   return_token_type_ids=True,
                                   return_attention_mask=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels

#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=2,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

print(len(loader))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels

1


(torch.Size([2, 500]),
 torch.Size([2, 500]),
 torch.Size([2, 500]),
 tensor([0, 1]))

In [63]:
from transformers import BertModel

#加载预训练模型
pretrained = RobertaModel.from_pretrained('roberta-base')

#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

#模型试算
out = pretrained(input_ids=input_ids,
           attention_mask=attention_mask,
           token_type_ids=token_type_ids)

out.last_hidden_state.shape

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([2, 500, 768])

In [64]:
#定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)

        out = self.fc(out.last_hidden_state[:, 0])

        out = out.softmax(dim=1)

        return out


model = Model()

model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape

torch.Size([2, 2])

In [65]:
from transformers import AdamW

#训练
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()

model.train()
for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    
    out = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if i % 5 == 0:
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)

        print(i, loss.item(), accuracy)

    if i == 300:
        break

/root/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 0.6916657090187073 0.5


In [68]:
#测试
def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=Dataset('test'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):

        if i == 5:
            break

        print(i)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

    print(correct / total)


test()

Using custom data configuration test-903f5bc504a53c34


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/test-903f5bc504a53c34/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.
0
1
2
3
4
0.53125
